In [1]:
import tensorflow as tf
import numpy as np
import zipfile
import urllib
import os

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Downloading the Zip
train_link = "https://storage.googleapis.com/learning-datasets/rps.zip"
train_file = "rps.zip"
urllib.request.urlretrieve(train_link, train_file)

# Extracting the Zip
train_dir = "Rock-Paper-Scissor/Training"
zip_ref_train = zipfile.ZipFile(train_file, "r")
zip_ref_train.extractall(train_dir)
zip_ref_train.close()

In [3]:
# Downloading the Zip
test_link = "https://storage.googleapis.com/learning-datasets/rps-test-set.zip"
test_file = "rps-test-set.zip"
urllib.request.urlretrieve(test_link, test_file)

# Extracting the Zip
test_dir = "Rock-Paper-Scissor/Testing"
zip_ref_test = zipfile.ZipFile(test_file, "r")
zip_ref_test.extractall(test_dir)
zip_ref_test.close()

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

test_datagen = ImageDataGenerator(
    rescale=1/255
)

In [5]:
train_gen = train_datagen.flow_from_directory(
    "/kaggle/working/Rock-Paper-Scissor/Training/rps/",
    class_mode="categorical",
    target_size=(150, 150),
)

Found 2520 images belonging to 3 classes.


In [6]:
test_gen = test_datagen.flow_from_directory(
    "/kaggle/working/Rock-Paper-Scissor/Testing/rps-test-set/",
    class_mode="categorical",
    target_size=(150, 150),
)

Found 372 images belonging to 3 classes.


In [7]:
tf.keras.backend.clear_session()

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(
    loss="categorical_crossentropy",
    metrics=["accuracy"],
    optimizer="rmsprop"
)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)       0

In [10]:
history = model.fit(
    train_gen, 
    validation_data=test_gen,
    epochs=20,
    verbose=1
)

Epoch 1/20
79/79 [==============================] - 27s 289ms/step - loss: 1.1644 - accuracy: 0.3448 - val_loss: 1.1852 - val_accuracy: 0.3360
Epoch 2/20
79/79 [==============================] - 23s 292ms/step - loss: 1.0580 - accuracy: 0.4798 - val_loss: 3.6201 - val_accuracy: 0.6667
Epoch 3/20
79/79 [==============================] - 23s 290ms/step - loss: 0.9808 - accuracy: 0.6361 - val_loss: 0.4246 - val_accuracy: 0.8280
Epoch 4/20
79/79 [==============================] - 23s 292ms/step - loss: 0.6040 - accuracy: 0.7425 - val_loss: 0.4309 - val_accuracy: 0.9247
Epoch 5/20
79/79 [==============================] - 23s 293ms/step - loss: 0.4709 - accuracy: 0.8282 - val_loss: 0.3448 - val_accuracy: 0.8575
Epoch 6/20
79/79 [==============================] - 23s 296ms/step - loss: 0.2925 - accuracy: 0.8881 - val_loss: 0.1532 - val_accuracy: 0.9328
Epoch 7/20
79/79 [==============================] - 23s 291ms/step - loss: 0.2519 - accuracy: 0.9115 - val_loss: 0.6150 - val_accuracy: 0.7930